# St. Louis Address Matching

Steps for setting up:

1. **Clone or download** this Github repository: [insert link]
2. **Download Conda.** The part of the code that uses ArcPy only works using the Conda distribution that comes with an ArcGIS Pro download (free for UCI affiliates, contact tsoeller@uci.edu). Everything else should work with any installation of Python 3: https://www.continuum.io/downloads.
3. **Download Python packages** that are not included with Anaconda, using the command-line as an administrator:
```
pip install lob
pip install pygeocoder
```

**Import packages** and set working directory

In [1]:
import pandas as pd
import csv
import datetime
import lob
import os
import sys
from pygeocoder import Geocoder
import arcpy
from arcpy import env
import numpy as np

os.chdir(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook')

ImportError: Missing required dependencies ['numpy']

# Standardize Addresses

The next 3 cells standardize lists of addresses using Lob's free API, so that the lists can be merged. The API checks against USPS's current list of valid postal addresses and returns the address in CASS format: https://en.wikipedia.org/wiki/Coding_Accuracy_Support_System.

The API returns about 2 addresses per second, so it can take a while. The output is stored in a *.csv, meaning you can skip this step. If you want to make sure it works without re-running the whole cell, run a few addresses and save to a different file. Do this in the first cell by replacing the first line with:
```
approved = pd.read_csv('Approved_StLouis_Cleaned.csv').head()
``` 
and the last line with:
```
approved.to_csv('delete.csv', index=False)
```

(1) **Addresses of approved applicants** (Takes a long time.)

In [ ]:
approved = pd.read_csv('Approved_StLouis_Cleaned.csv') # Import .csv

lob.api_key = 'test_fc26575412e92e22a926bc96c857f375f8b'

for index, row in approved.iterrows():
    try:    
        verifiedAddress = lob.Verification.create(
            address_line1=row['street'],
            address_city=row['city'],
            address_state=row['state'],
            address_zip=row['zip'],
            address_country='US'
        )
        approved.ix[approved.index == index, 'lob_addr1'] = verifiedAddress.address.address_line1
        approved.ix[approved.index == index, 'lob_addr2'] = verifiedAddress.address.address_line2
        approved.ix[approved.index == index, 'lob_city'] = verifiedAddress.address.address_city
        approved.ix[approved.index == index, 'lob_zip'] = verifiedAddress.address.address_zip               
    except:
        approved.ix[approved.index == index, 'lob_addr1'] = "no"
        approved.ix[approved.index == index, 'lob_addr2'] = "no"
        approved.ix[approved.index == index, 'lob_city'] = "no"
        approved.ix[approved.index == index, 'lob_zip'] = "no"       
                   
approved.to_csv('lob_approved.csv', index=False)

(2) **Addresses of denied applicants**  (Takes a long time.)

In [ ]:
denied = pd.read_csv('Denied_StLouis_Cleaned.csv')

lob.api_key = 'test_fc26575412e92e22a926bc96c857f375f8b'

for index, row in denied.iterrows():
    try:    
        verifiedAddress = lob.Verification.create(
            address_line1=row['street'],
            address_city=row['city'],
            address_state=row['state'],
            address_zip=row['zip'],
            address_country='US'
        )
        denied.ix[denied.index == index, 'lob_addr1'] = verifiedAddress.address.address_line1
        denied.ix[denied.index == index, 'lob_addr2'] = verifiedAddress.address.address_line2
        denied.ix[denied.index == index, 'lob_city'] = verifiedAddress.address.address_city
        denied.ix[denied.index == index, 'lob_zip'] = verifiedAddress.address.address_zip               
    except:
        denied.ix[denied.index == index, 'lob_addr1'] = "no"
        denied.ix[denied.index == index, 'lob_addr2'] = "no"
        denied.ix[denied.index == index, 'lob_city'] = "no"
        denied.ix[denied.index == index, 'lob_zip'] = "no"       
                   
denied.to_csv('lob_denied.csv', index=False)

(3) **County assessor's list of land parcels associated to postal addresses.** Each land parcel may have multiple postal addresses but has a unique "handle".  (Takes a long time.)

In [ ]:
# Address to Parcel Mapping by St Louis city govt
a2p = pd.read_csv('addr_to_parcel.csv')

a2p['street'] = a2p['HOUSENUM'].fillna('').astype(str) + " " + a2p['HOUSESUF'].fillna('').astype(str)
a2p['street'] = a2p['street'].str.rstrip(" ")

a2p['street'] = a2p['street'].fillna('').astype(str) + " " + a2p['PREDIR'].fillna('').astype(str)
a2p['street'] = a2p['street'].str.rstrip(" ")

a2p['street'] = a2p['street'].fillna('').astype(str) + " " + a2p['STREETNAME'].fillna('').astype(str)
a2p['street'] = a2p['street'].str.rstrip(" ")

a2p['street'] = a2p['street'].fillna('').astype(str) + " " + a2p['STREETTYPE'].fillna('').astype(str)
a2p['street'] = a2p['street'].str.rstrip(" ")

a2p['street'] = a2p['street'].fillna('').astype(str) + " " + a2p['SUFDIR'].fillna('').astype(str)
a2p['street'] = a2p['street'].str.rstrip(" ")

a2p['street'] = a2p['street'].fillna('').astype(str) + " " + a2p['UNITNUM'].fillna('').astype(str)
a2p['street'] = a2p['street'].str.rstrip(" ")

for index, row in a2p.iterrows():
    try:    
        verifiedAddress = lob.Verification.create(
            address_line1=row['street'],
            address_city="Saint Louis",
            address_state="MO",
            address_country='US'
        )
        a2p.ix[a2p.index == index, 'lob_addr1'] = verifiedAddress.address.address_line1
        a2p.ix[a2p.index == index, 'lob_addr2'] = verifiedAddress.address.address_line2
        a2p.ix[a2p.index == index, 'lob_city'] = verifiedAddress.address.address_city
        a2p.ix[a2p.index == index, 'lob_zip'] = verifiedAddress.address.address_zip               
    except:
        a2p.ix[a2p.index == index, 'lob_addr1'] = "no"
        a2p.ix[a2p.index == index, 'lob_addr2'] = "no"
        a2p.ix[a2p.index == index, 'lob_city'] = "no"
        a2p.ix[a2p.index == index, 'lob_zip'] = "no"       

                   
a2p.to_csv('lob_a2p.csv', index=False)

# Merge applicant addresses to city parcels

Now that addresses are in a standard format, <strong>merge applicant addresses</strong> (both approved and denied) to a unique parcel "handle" using the St Louis city address list. 

We first make some adjustments to maximize the number of addresses that will merge successfully: 
1. Remove apartment numbers from applicant addresses since parcels never have an apartment number.
2. The API doesn't return a result for some applicant addresses, meaning they weren't recognized as valid postal addresses. Reasons include (1) the address is incomplete or written incorrectly, (2) the house is vacant, so not an address where USPS delivers, or (3) the application is for an address that doesn't exist yet, in anticipation of a parcel split. For this reason, create a combined column with the standardized address if there is one and the original address if the API returned no result.

This cell generates a lot of warnings, but it runs. Addresses that fail to merge in this cell are dealt with later.

In [ ]:
approved = pd.read_csv('lob_approved.csv')
# Remove apartment numbers
approved['lob_addr1'] = approved['lob_addr1'].apply(lambda x: x.split(' #')[0])
approved['street'] = approved['street'].apply(lambda x: x.split(' #')[0])

denied = pd.read_csv('lob_denied.csv')
# Remove apartment numbers
denied['lob_addr1'] = denied['lob_addr1'].apply(lambda x: x.split(' #')[0])
denied['street'] = denied['street'].apply(lambda x: x.split(' #')[0])

a2p = pd.read_csv('lob_a2p.csv') # a2p means "address to parcel"

# Create combined address column
approved['best_addr1'] = approved['lob_addr1']
mask = approved['best_addr1']=='no'
approved['best_addr1'][mask] = approved['street'].str.upper()
approved['best_addr1'] = approved['best_addr1'].str.replace('.','')
approved['best_city'] = approved['lob_city']
approved['best_city'][mask] = approved['city'].str.upper()
approved['best_zip'] = approved['lob_zip']
approved['best_zip'][mask] = approved['zip']

# Create combined address column
denied['best_addr1'] = denied['lob_addr1']
mask = denied['best_addr1']=='no'
denied['best_addr1'][mask] = denied['street'].str.upper()
denied['best_addr1'] = denied['best_addr1'].str.replace('.','')
denied['best_addr1'] = denied['best_addr1'].str.replace('STREET','ST')
denied['best_addr1'] = denied['best_addr1'].str.replace('AVENUE','AVE')
denied['best_city'] = denied['lob_city']
denied['best_city'][mask] = denied['city'].str.upper()
denied['best_zip'] = denied['lob_zip']
denied['best_zip'][mask] = denied['zip']

# Create combined address column
a2p['best_addr1'] = a2p['lob_addr1']
mask = a2p['best_addr1']=='no'
a2p['best_addr1'][mask] = a2p['street'].str.upper()
a2p['best_city'] = a2p['lob_city']
a2p['best_city'][mask] = 'SAINT LOUIS'
a2p['best_zip'] = a2p['lob_zip']
a2p['best_zip'][mask] = ''
a2p_lob = a2p[['HANDLE', 'best_addr1', 'best_city']]
a2p_lob = a2p_lob.drop_duplicates(['best_addr1', 'best_city'])

# Perform the actual merges
approved = approved.merge(a2p_lob,left_on=['best_addr1', 'best_city'], right_on=['best_addr1', 'best_city'], how='left')
denied = denied.merge(a2p_lob,left_on=['best_addr1', 'best_city'], right_on=['best_addr1', 'best_city'], how='left')

# Save to .csv
approved.to_csv('approved_matched.csv')
denied.to_csv('denied_matched.csv')

Create a combined **list of all the addresses that failed to merge** with a parcel located in St Louis city limits

In [ ]:
unmatched_approved = approved[approved['HANDLE'].isnull()][['lob_addr1', 'lob_city', 'lob_zip', 'HANDLE', 'best_addr1', 'best_city', 'best_zip']]
unmatched_approved['source'] = "approved"
unmatched_denied = denied[denied['HANDLE'].isnull()][['lob_addr1', 'lob_city', 'lob_zip', 'HANDLE', 'best_addr1', 'best_city', 'best_zip']]
unmatched_denied['source'] = "denied"

unmatched = unmatched_approved.append(unmatched_denied)
unmatched['best_address'] = unmatched['best_addr1'].astype(str) + ", " + unmatched['best_city'].astype(str) + ", MO " + unmatched['best_zip'].astype(str)
unmatched_address = unmatched[['best_address']].drop_duplicates()


**Fix** a few specific **typos** and quirks.

In [ ]:
unmatched_address['best_address'] = unmatched_address['best_address'].str.replace('GLENECHORT','GLENCHORT')
unmatched_address['best_address'] = unmatched_address['best_address'].str.replace('PARKER GRAY','PARKERGREY')
unmatched_address['best_address'] = unmatched_address['best_address'].str.replace('1720 SOUTH 14TH ST','1720 S 14TH ST')
unmatched_address['best_address'] = unmatched_address['best_address'].apply(lambda x: x.split(' REAR')[0])
unmatched_address['best_address'] = unmatched_address['best_address'].apply(lambda x: x.split(' FRNT')[0])

**Geocode** addresses of unmatched addresses, using Google's Geocoding API. (This takes a long time.)

The first 2,500 addresses geocoded in a given day are free. Any more are billed at $1 per 1,000 geocodes. This step geocodes 1,841 addresses. Thus, you'll need to insert an API key to run this more than once in a day. That is replace the 3rd line with:
```
results = Geocoder('<<YOUR KEY>>').geocode(address)
```

You can sign up for a Google Geocoding API key at https://developers.google.com/maps/documentation/geocoding/start. [Matt, I'll email you mine. I just don't want it up on Github.]

[Matt, FYI, we might technically be violating Google Geocoding API's terms of service, which say "The Google Maps Geocoding API may only be used in conjunction with displaying results on a Google map. It is prohibited to use Google Maps Geocoding API data without displaying a Google map." We can use another geocoding API if it ever becomes a problem.]

In [ ]:
def Google_geocode(address):
    try:
        results = Geocoder.geocode(address)  # API key with payment method required for +2500 queries per day
        return results.coordinates
        print(index)
    except:
        return (0.0, 0.0)
unmatched_address['best_lat'], unmatched_address['best_lon'] = zip(*unmatched_address['best_address'].apply(Google_geocode))
unmatched_address.to_csv('unmatched_address.csv', index=False)

Only an upduplicated list of unmatched addresses is geocoded, to keep the process "cheap". After geocoding, rematch to the full list of unmatched applications, which contains some duplicate addresses.

In [ ]:
unmatched_address = pd.read_csv('unmatched_address.csv')
unmatched = unmatched.merge(unmatched_address,left_on='best_address', right_on='best_address', how='left')
unmatched.to_csv('unmatched.csv')

Overlay geocodes of unmatched addresses and the assessor's parcel map in ArcGIS. Use these to perform matches, described later. 

Running this cell requires the ArcPy package, which comes in the Conda installation included with ArcGIS Pro. You have to be signed in to ArcGIS Pro via the web to run ArcPy. You would also have to change the filepaths to run this cell. [Open source software could do this. I just happen to work in ArcGIS. We can change this part later if needed.]

In [ ]:
    # Set ArcPy environment settings
arcpy.env.workspace = r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb'
arcpy.env.overwriteOutput = True
    # Add points to ArcGIS as a Table and display them on the map
        #TableToTable_conversion (in_rows, out_path, out_name, {where_clause}, {field_mapping}, {config_keyword})
arcpy.TableToTable_conversion(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\unmatched.csv', r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb', "unmatched6")
        #MakeXYEventLayer_management(in_Table, x_coords, y_coords, out_Layer, spRef, z_coords)
arcpy.management.MakeXYEventLayer("unmatched6", "best_lon", "best_lat", "unmatched", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
    # Add St Louis city limits
        #CopyFeatures_management (in_features, out_feature_class, {config_keyword}, {spatial_grid_1}, {spatial_grid_2}, {spatial_grid_3})
arcpy.management.CopyFeatures(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\shapefiles\stl_boundary.shp', 'stl_boundary', None, None, None, None)
arcpy.management.CopyFeatures(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\shapefiles\prcl.shp', 'prcl', None, None, None, None)
    # Find which points fall inside the city using a spatial join
        #SpatialJoin_analysis (target_features, join_features, out_feature_class, {join_operation}, {join_type}, {field_mapping}, {match_option}, {search_radius}, {distance_field_name})
arcpy.analysis.SpatialJoin("unmatched", "stl_boundary", r"C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb\unmatched_SpatialJoin", "JOIN_ONE_TO_MANY", "KEEP_ALL", 'lob_addr1 "lob_addr1" true true false 8000 Text 0 0,First,#,unmatched3_Layer,lob_addr1,0,8000;lob_city "lob_city" true true false 8000 Text 0 0,First,#,unmatched3_Layer,lob_city,0,8000;lob_zip "lob_zip" true true false 8000 Text 0 0,First,#,unmatched3_Layer,lob_zip,0,8000;HANDLE "HANDLE" true true false 8000 Text 0 0,First,#,unmatched3_Layer,HANDLE,0,8000;best_addr1 "best_addr1" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_addr1,0,8000;best_city "best_city" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_city,0,8000;best_zip "best_zip" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_zip,0,8000;source "source" true true false 8000 Text 0 0,First,#,unmatched3_Layer,source,0,8000;best_address "best_address" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_address,0,8000;best_lat "best_lat" true true false 8 Double 0 0,First,#,unmatched3_Layer,best_lat,-1,-1;best_lon "best_lon" true true false 8 Double 0 0,First,#,unmatched3_Layer,best_lon,-1,-1;LAYER "LAYER" true true false 32 Text 0 0,First,#,stl_boundary,LAYER,0,32', "INTERSECT", None, None)
arcpy.analysis.SpatialJoin("unmatched_SpatialJoin", "prcl", r"C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb\unmatched_SpatialJoin1", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'lob_addr1 "lob_addr1" true true false 8000 Text 0 0,First,#,unmatched3_Layer,lob_addr1,0,8000;lob_city "lob_city" true true false 8000 Text 0 0,First,#,unmatched3_Layer,lob_city,0,8000;lob_zip "lob_zip" true true false 8000 Text 0 0,First,#,unmatched3_Layer,lob_zip,0,8000;HANDLE "HANDLE" true true false 8000 Text 0 0,First,#,unmatched3_Layer,HANDLE,0,8000;best_addr1 "best_addr1" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_addr1,0,8000;best_city "best_city" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_city,0,8000;best_zip "best_zip" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_zip,0,8000;source "source" true true false 8000 Text 0 0,First,#,unmatched3_Layer,source,0,8000;best_address "best_address" true true false 8000 Text 0 0,First,#,unmatched3_Layer,best_address,0,8000;best_lat "best_lat" true true false 8 Double 0 0,First,#,unmatched3_Layer,best_lat,-1,-1;best_lon "best_lon" true true false 8 Double 0 0,First,#,unmatched3_Layer,best_lon,-1,-1;LAYER "LAYER" true true false 32 Text 0 0,First,#,stl_boundary,LAYER,0,32;HANDLE_1 "HANDLE" true true false 16 Text 0 0,First,#,prcl,HANDLE,0,16', "INTERSECT", None, None)

arcpy.stats.ExportXYv("unmatched_SpatialJoin1", "lob_addr1;lob_city;lob_zip;HANDLE;HANDLE_1;best_addr1;best_city;best_zip;source;best_address;best_lat;best_lon;LAYER", "SEMI-COLON", r"C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\unmatched_vCityLimits_ArcExport.csv", "ADD_FIELD_NAMES")

The previous cell made an overlay of applicant addresses and parcels. Use this overlay to resolve unmatched addresses as follows: 
1. Geocoded applicant addresses that overlap a parcel are assigned to that parcel. 
2. Geocoded applications that are outside of St. Louis city limits are thrown out. 
3. Records that weren't geocoded are set to a missing handle value and dealt with later.
3. Geocodes are either to the roof top or to the street center line. Geocodes to the street center line don't overlap any parcel. The resolution of these cases is explained below. 

Create a variable to start keeping track of how matches are made.

In [ ]:
unmatched = pd.read_csv('unmatched_vCityLimits_ArcExport.csv',sep=';')
unmatched['match_type'] = np.nan

# Apply the handle assigned by ArcGIS, subject to 3 caveats in the lines that follow
unmatched['HANDLE'] = unmatched['HANDLE_1']
unmatched['match_type'][unmatched['HANDLE_1'].notnull()] = 'GIS_overlay_insideSTL'

mask = unmatched['LAYER'].isnull() # Set records not found to be in St. Louis city limits to handle value 9999 (un-geocoded are modified by the next condition)
unmatched['HANDLE'][mask] = 9999 
unmatched['match_type'][mask] = 'GIS_overlay_outsideSTL'

mask = unmatched['XCoord']==0 # Set records that weren't geocoded to a missing handle value
unmatched['HANDLE'][mask] = np.nan 
unmatched['match_type'][mask] = np.nan

This is the first place things get messy. 

About 70 applications list multiple street addresses (e.g., "4560 & 4562 Oak Ln"). I manually matched the relevant parcels to these applications in a spreadsheet, based on visual inspection of ArcGIS. The maximum number of parcels for one application is 4, listed in 4 separate columns. This cell adds these matches to the larger list.

Continue keeping track of how matches are made using the <i>match_type</i> column.

In [ ]:
multi = pd.read_csv('multi.csv', index_col="INDEX")
unmatched = unmatched.merge(multi, left_index=True, right_index=True, how='left')

mask = unmatched['HANDLE_M1'].notnull()
unmatched['match_type'][mask] = 'manual_multi'

unmatched['HANDLE_M1'][~mask] = unmatched['HANDLE']

About 400 applications are geocoded to the street center line and don't overlap any parcel. Instead, these addresses are assigned to (up to) two nearby parcels on the same block face: the parcel with the next highest house number and the parcel with the next lowest house number. The working assumption is that the applicant was thinking of building on one of these two parcels.

Finding houses above and below on the same block face required separating street name from house number. This was done by hand in Excel.

Several address typos are corrected before doing this match. If the "fixed" application address now matches a parcel address exactly, only that one parcel is assigned.

(Takes ~2 minutes.)

In [ ]:
# For addresses with no matching parcel address that are geocoded to the street centerline, match to the parcels above and below it on the same size of the street and on the same block
    # Create csv in Excel by hand to isolate street name
between = pd.read_csv('between.csv') 
    # Fix some problems in the addresses
between['STREET_NAME'] = between['STREET_NAME'].str.replace('CRANE', 'CRANE CIRCLE')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('FARIMONT', 'FAIRMOUNT')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('HERBERT', 'HEBERT')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('KENSINGSTON', 'KENSINGTON')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('LOVELOY', 'LOVEJOY')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('MALLINCRODT', 'MALLINCKRODT')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('MARYLAND,', 'MARYLAND')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('MONGTGOMERY', 'MONTGOMERY')
between['STREET_NAME'] = between['STREET_NAME'].str.replace('PRAIRE', 'PRAIRIE')  
between['STREET_NAME'] = between['STREET_NAME'].str.replace('REV G H PRUITT', 'REV GEORGE H PRUITT')                     
between['STREET_NAME'] = between['STREET_NAME'].str.replace('SAINT FERDINAND', 'ST FERDINAND')                     
between['STREET_NAME'] = between['STREET_NAME'].str.replace('SAINT LOUIS', 'ST LOUIS')                     
between['STREET_NAME'] = between['STREET_NAME'].str.replace('SAINT VINCENT', 'VINCENT')                     
between['STREET_NAME'] = between['STREET_NAME'].str.replace('VINCENT', 'ST VINCENT')  
between['STREET_NAME'] = between['STREET_NAME'].str.replace('WINSOR', 'WINDSOR')                     
between['STREET_NAME'] = between['STREET_NAME'].str.replace('WASHINGON', 'WASHINGTON')                      
    # Merge isolated street name into the larger list of unmatched addresses
unmatched['INDEX'] = unmatched.index
unmatched = unmatched.merge(between, left_on='INDEX', right_on='INDEX', how='left')
    # Create variables to designate side of street and block number
a2p['EVEN'] = 0
a2p['EVEN'][a2p['HOUSENUM'].astype(int) % 2 == 0] = 1
a2p['EVEN'] = a2p['EVEN'].astype(float)
a2p['BLOCK'] = a2p['HOUSENUM'].astype(float).floordiv(100)
a2p['STREETNAME'] = a2p['PREDIR'].fillna('').astype(str) + " " + a2p['STREETNAME']
a2p['STREETNAME'] = a2p['STREETNAME'].str.lstrip(" ")
a2p = a2p[a2p['HANDLE']!=12022000010] # 1,200 addresses are all assigned to the center of a city park (ie, parcel 12022000010), so I'm throwing it out.
unmatched['BLOCK'] = unmatched['HOUSE'].floordiv(100)
unmatched['HIGH'] = np.nan
unmatched['LOW'] = np.nan

In [ ]:
# For loops to find address immediately below and above and the corresponding parcel. (Keep separate so that 1 can succeed even if the other finds nothing.)
for index, row in unmatched.iterrows():
    try:
        samestreet = a2p.STREETNAME==row['STREET_NAME']
        sameblock = a2p.BLOCK==row['BLOCK']
        evenOdd = a2p.EVEN==row['EVEN']
        lower = a2p.HOUSENUM<=row['HOUSE']
        a2p_lower = a2p[samestreet][sameblock][evenOdd][lower][::-1].reset_index() 
        unmatched.ix[unmatched.index == index, 'house_LOW'] = a2p_lower.ix[0,'HOUSENUM']
        unmatched.ix[unmatched.index == index, 'street_LOW'] = a2p_lower.ix[0,'STREETNAME']
        unmatched.ix[unmatched.index == index, 'handle_LOW'] = a2p_lower.ix[0,'HANDLE']
    except:
        unmatched.ix[unmatched.index == index, 'LOW'] = np.nan
        unmatched.ix[unmatched.index == index, 'street_LOW'] = np.nan
        unmatched.ix[unmatched.index == index, 'handle_LOW'] = np.nan            

In [ ]:
for index, row in unmatched.iterrows():
    try:
        samestreet = a2p.STREETNAME==row['STREET_NAME']
        sameblock = a2p.BLOCK==row['BLOCK']
        evenOdd = a2p.EVEN==row['EVEN']
        higher = a2p.HOUSENUM>=row['HOUSE']
        a2p_higher = a2p[samestreet][sameblock][evenOdd][higher].reset_index() 
        unmatched.ix[unmatched.index == index, 'house_HIGH'] = a2p_higher.ix[0,'HOUSENUM']
        unmatched.ix[unmatched.index == index, 'street_HIGH'] = a2p_higher.ix[0,'STREETNAME']
        unmatched.ix[unmatched.index == index, 'handle_HIGH'] = a2p_higher.ix[0,'HANDLE']
    except:
        unmatched.ix[unmatched.index == index, 'HIGH'] = np.nan
        unmatched.ix[unmatched.index == index, 'street_HIGH'] = np.nan
        unmatched.ix[unmatched.index == index, 'handle_HIGH'] = np.nan    

In [ ]:
mask = unmatched['STREET_NAME'].notnull()     
unmatched['match_type'][mask] = 'between'

unmatched['HANDLE_M1'][mask] = unmatched['handle_LOW']
unmatched['HANDLE_M1'][mask & unmatched['handle_LOW'].isnull()] = unmatched['handle_HIGH']
unmatched['HANDLE_M2'][mask & unmatched['handle_LOW'].notnull()] = unmatched['handle_HIGH']
unmatched['HANDLE_M2'][unmatched['HANDLE_M1']==unmatched['HANDLE_M2']] = np.nan
unmatched['HANDLE_M1'][mask & unmatched['HANDLE_M1'].isnull()] = 7777

The few remaining addresses were resolved by hand. 9999 is assigned if the address is outside city limits. 7777 is assigned if the address cannot be found.

In [ ]:
# Assign handles to the last stragglers by hand using visual inspection in ArcGIS 
unmatched['HANDLE_M1'][unmatched['BEST_ADDR1'].str.contains(pat='DOMBARD LN')] = 9999
unmatched['match_type'][unmatched['BEST_ADDR1'].str.contains(pat='DOMBARD LN')] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='444 N VAN BUREN AVE'] = 9999
unmatched['match_type'][unmatched['BEST_ADDR1']=='444 N VAN BUREN AVE'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='9807 LAWNVIEW DR'] = 9999
unmatched['match_type'][unmatched['BEST_ADDR1']=='9807 LAWNVIEW DR'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='SALISBURY PARK HOMES - BUILDING LOT'] = 7777
unmatched['match_type'][unmatched['BEST_ADDR1']=='SALISBURY PARK HOMES - BUILDING LOT'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='LOT 3 CABANNE AVE'] = 7777
unmatched['match_type'][unmatched['BEST_ADDR1']=='LOT 3 CABANNE AVE'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='6334 BISHOP PLACE'] = 7777
unmatched['match_type'][unmatched['BEST_ADDR1']=='6334 BISHOP PLACE'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='ONE MORGAN ST'] = 7777
unmatched['match_type'][unmatched['BEST_ADDR1']=='ONE MORGAN ST'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='SALISBURY PARK HOMES - BUILDING LOT'] = 7777   
unmatched['match_type'][unmatched['BEST_ADDR1']=='SALISBURY PARK HOMES - BUILDING LOT'] = 'manual_straggler'   

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='6502 W PARK AVE'] = 14893000100
unmatched['match_type'][unmatched['BEST_ADDR1']=='6502 W PARK AVE'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='18 ARBERDEEN'] = 15639000020     
unmatched['match_type'][unmatched['BEST_ADDR1']=='18 ARBERDEEN'] = 'manual_straggler'     

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='3625 FLAD HOUSE'] = 12118000120
unmatched['match_type'][unmatched['BEST_ADDR1']=='3625 FLAD HOUSE'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='1212 REV G H PRUITT PL'] = 14553050450
unmatched['match_type'][unmatched['BEST_ADDR1']=='1212 REV G H PRUITT PL'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='1927 SAINT LOUIS AVE'] = 11104000180
unmatched['match_type'][unmatched['BEST_ADDR1']=='1927 SAINT LOUIS AVE'] = 'manual_straggler'

unmatched['HANDLE_M2'][unmatched['BEST_ADDR1']=='1927 SAINT LOUIS AVE'] = 11104000190
unmatched['match_type'][unmatched['BEST_ADDR1']=='1927 SAINT LOUIS AVE'] = 'manual_straggler'

unmatched['HANDLE_M1'][unmatched['BEST_ADDR1']=='4140 DE TONTY ST'] = 15310000125
unmatched['match_type'][unmatched['BEST_ADDR1']=='4140 DE TONTY ST'] = 'manual_straggler'

unmatched['HANDLE_M2'][unmatched['BEST_ADDR1']=='4140 DE TONTY ST'] = 15310000135
unmatched['match_type'][unmatched['BEST_ADDR1']=='4140 DE TONTY ST'] = 'manual_straggler'

Recent cells have focused on resolving the parcel for applications that didn't merge correctly at first. Add the handle assignments for these "trouble" applications back into the main lists of approved and denied applications. Also, combine approved and denied applications into one list.

In [ ]:
# Append the approved and denied lists, keeping track of reference numbers
approved['refnum'] = approved['proj_num']
matched = approved[['year', 'authorized', 'issued', 'owner', 'best_addr1', 'best_zip', 'HANDLE', 'refnum']]
matched['result'] = 'approved'
denied['result'] = denied['application']
temp = denied[['year', 'result', 'best_addr1', 'best_zip', 'HANDLE', 'refnum']]
matched = matched.append(temp, ignore_index=True)
matched['best_addr1'] = matched['best_addr1'].astype(str)
matched['best_zip'] = matched['best_zip'].astype(str)

# Merge in handles for those that didn't match at first
unmatched['best_addr1'] = unmatched['BEST_ADDR1'].astype(str)
unmatched['best_zip'] = unmatched['BEST_ZIP'].astype(str)
#unmatched['HANDLE_M1'] = unmatched['HANDLE']
temp = unmatched[['best_addr1', 'best_zip', 'match_type', 'HANDLE_M1', 'HANDLE_M2', 'HANDLE_M3', 'HANDLE_M4']].drop_duplicates(['best_addr1', 'best_zip'])
matched = matched.merge(temp, left_on=['best_addr1', 'best_zip'], right_on=['best_addr1', 'best_zip'], how='left')
matched['HANDLE_M1'][matched['HANDLE'].notnull()] = matched['HANDLE']
matched = matched.drop('HANDLE', axis=1)

# Assign match type to applications that matched nicely at first
mask = matched['match_type'].isnull()
matched['match_type'][mask] = 'clean_match'

**Export the final list** of handle assignments for all applications to a CSV.

In [ ]:
matched.to_csv('final_handle_assignments.csv', index=False)

# Summary statistics of matches

Display a few **summary statistics**: 
1. Number of approved and denied applications, by type of match
3. Number of applications assigned to multiple parcels
4. Number of applications thrown out because they are outside city limits ("9999") or were not found ("7777")

In [ ]:
pd.crosstab(matched['match_type'], matched['result'], margins=True)

In [ ]:
matched[['HANDLE_M1', 'HANDLE_M2', 'HANDLE_M3', 'HANDLE_M4']].info()

In [ ]:
summary = matched[['result', 'HANDLE_M1', 'match_type']][matched['HANDLE_M1'].isin([7777, 9999])]
pd.crosstab(summary['result'], summary['HANDLE_M1'], margins=True)
# 9999 is outside city limits, 7777 are applications with addresses that couldn't be found

In [ ]:
pd.crosstab(summary['match_type'], summary['HANDLE_M1'], margins=True)
# 9999 is outside city limits, 7777 are applications with addresses that couldn't be found

# Illustrative maps of results

**Map paths** from applicant addresses that didn't match initially to the parcels they were eventually matched with. This is a way to check results.

First parcel match for each application (PARCEL_M1):

In [ ]:
pargeocd = pd.read_csv('pargeocd.csv', sep = ";")
pargeocd = pargeocd.drop_duplicates('HANDLE')
trouble_map = unmatched.merge(pargeocd,left_on='HANDLE_M1', right_on='HANDLE', how='inner')
trouble_map = trouble_map[['XCoord_x', 'YCoord_x', 'BEST_ADDRESS', 'match_type', 'HANDLE_M1', 'HANDLE_M2', 'HANDLE_M3', 'HANDLE_M4', 'XCoord_y', 'YCoord_y', 'HANDLE_y']]
trouble_map['XCoord_x'][trouble_map['match_type']=='manual_multi'] = np.nan
trouble_map['YCoord_x'][trouble_map['match_type']=='manual_multi'] = np.nan
trouble_map['XCoord_x'][trouble_map['YCoord_x']==0] = np.nan
trouble_map['YCoord_x'][trouble_map['YCoord_x']==0] = np.nan
trouble_map.to_csv('trouble_map_M1.csv')

In [ ]:
    # Set ArcPy environment settings
arcpy.env.workspace = r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb'
arcpy.env.overwriteOutput = True
    # Add points to ArcGIS as a Table and display them on the map
        #TableToTable_conversion (in_rows, out_path, out_name, {where_clause}, {field_mapping}, {config_keyword})
arcpy.TableToTable_conversion(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\trouble_map_M1.csv', r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb', "trouble_M1")
        #MakeXYEventLayer_management(in_Table, x_coords, y_coords, out_Layer, spRef, z_coords)
arcpy.management.MakeXYEventLayer("trouble_M1", "XCoord_x", "YCoord_x", 'trouble_geocodes_layer_M1', "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
arcpy.management.MakeXYEventLayer("trouble_M1", "XCoord_y", "YCoord_y", 'trouble_parcels_layer_M1', "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
        #CopyFeatures_management (in_features, out_feature_class, {config_keyword}, {spatial_grid_1}, {spatial_grid_2}, {spatial_grid_3})
arcpy.management.CopyFeatures('trouble_geocodes_layer_M1', 'geocodes_M1', None, None, None, None)
arcpy.management.CopyFeatures('trouble_parcels_layer_M1', 'parcels_M1', None, None, None, None)
arcpy.management.CopyFeatures(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\shapefiles\prcl.shp', 'prcl', None, None, None, None)
arcpy.management.CopyFeatures(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\shapefiles\stl_boundary.shp', 'stl_boundary', None, None, None, None)
arcpy.management.CopyFeatures(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\shapefiles\pargeocd.shp', 'prcl_pts', None, None, None, None)
    # Draw line between geocodes and parcels
        #XYToLine_management (in_table, out_featureclass, startx_field, starty_field, endx_field, endy_field, {line_type}, {id_field}, {spatial_reference})
arcpy.management.XYToLine("trouble_M1", "trouble_path_M1", "XCoord_x", "YCoord_x", "XCoord_y", "YCoord_y", "GEODESIC", "HANDLE_M1", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")

Second parcel match (PARCEL_M2):

In [ ]:
pargeocd = pd.read_csv('pargeocd.csv', sep = ";")
pargeocd = pargeocd.drop_duplicates('HANDLE')
trouble_map = unmatched.merge(pargeocd,left_on='HANDLE_M2', right_on='HANDLE', how='inner')
trouble_map = trouble_map[['XCoord_x', 'YCoord_x', 'BEST_ADDRESS', 'match_type', 'HANDLE_M1', 'HANDLE_M2', 'HANDLE_M3', 'HANDLE_M4', 'XCoord_y', 'YCoord_y', 'HANDLE_y']]
trouble_map['XCoord_x'][trouble_map['match_type']=='manual_multi'] = np.nan
trouble_map['YCoord_x'][trouble_map['match_type']=='manual_multi'] = np.nan
trouble_map['XCoord_x'][trouble_map['YCoord_x']==0] = np.nan
trouble_map['YCoord_x'][trouble_map['YCoord_x']==0] = np.nan
trouble_map.to_csv('trouble_map_M2.csv')

In [ ]:
    # Set ArcPy environment settings
arcpy.env.workspace = r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb'
arcpy.env.overwriteOutput = True
    # Add points to ArcGIS as a Table and display them on the map
        #TableToTable_conversion (in_rows, out_path, out_name, {where_clause}, {field_mapping}, {config_keyword})
arcpy.TableToTable_conversion(r'C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\trouble_map_M2.csv', r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb', "trouble_M2")
        #MakeXYEventLayer_management(in_Table, x_coords, y_coords, out_Layer, spRef, z_coords)
arcpy.management.MakeXYEventLayer("trouble_M2", "XCoord_x", "YCoord_x", 'trouble_geocodes_layer_M2', "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
arcpy.management.MakeXYEventLayer("trouble_M2", "XCoord_y", "YCoord_y", 'trouble_parcels_layer_M2', "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
        #CopyFeatures_management (in_features, out_feature_class, {config_keyword}, {spatial_grid_1}, {spatial_grid_2}, {spatial_grid_3})
arcpy.management.CopyFeatures('trouble_geocodes_layer_M2', 'geocodes_M2', None, None, None, None)
arcpy.management.CopyFeatures('trouble_parcels_layer_M2', 'parcels_M2', None, None, None, None)
    # Draw line between geocodes and parcels
        #XYToLine_management (in_table, out_featureclass, startx_field, starty_field, endx_field, endy_field, {line_type}, {id_field}, {spatial_reference})
arcpy.management.XYToLine("trouble_M2", "trouble_path_M2", "XCoord_x", "YCoord_x", "XCoord_y", "YCoord_y", "GEODESIC", "HANDLE_M2", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")

The preceding cells produce the following maps. The full maps can be browsed in a map viewing application.

The first map shows the paths between geocodes of applicant addresses with no exact match that fall on the street center line and the parcels to which they have been matched (ie, match_type=='between'). This picture demonstrates that the algorithm succeeds in matching these applicants to nearby parcels, because the paths are short. The second map zooms in on a section and displays the parcels.

Note that applicant addresses that couldn't be geocoded are excluded. (That includes all the match_type=='manual_multi'.)
<img src="shapefiles/between_paths.PNG">
<img src="shapefiles/between_paths_inset2.PNG">


Now show geocodes of applications that were thrown out for being (1) outside city limits or (2) unable to be found.

In [ ]:
excluded_9999 = unmatched[['XCoord', 'YCoord', 'BEST_ADDRESS', 'match_type', 'HANDLE_M1']][unmatched['HANDLE_M1']==9999]
excluded_7777 = unmatched[['XCoord', 'YCoord', 'BEST_ADDRESS', 'match_type', 'HANDLE_M1']][unmatched['HANDLE_M1']==7777]
excluded_9999.to_csv('excluded_9999.csv')
excluded_7777.to_csv('excluded_7777.csv')

    # Set ArcPy environment settings
arcpy.env.workspace = r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb'
arcpy.env.overwriteOutput = True
        #MakeXYEventLayer_management(in_Table, x_coords, y_coords, out_Layer, spRef, z_coords)
arcpy.management.MakeXYEventLayer(r"C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\excluded_9999.csv", "XCoord", "YCoord", "excluded_9999_Layer", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
arcpy.management.MakeXYEventLayer(r"C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\excluded_7777.csv", "XCoord", "YCoord", "excluded_7777_Layer", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
        #CopyFeatures_management (in_features, out_feature_class, {config_keyword}, {spatial_grid_1}, {spatial_grid_2}, {spatial_grid_3})
arcpy.management.CopyFeatures('excluded_9999_Layer', 'excluded_9999_Feature', None, None, None, None)
arcpy.management.CopyFeatures('excluded_7777_Layer', 'excluded_7777_Feature', None, None, None, None)

Blue dots are unable to be matched to a parcel and tan dots are outside city limits.
<img src="shapefiles/excluded.PNG">

Show all the parcels to which applicant addresses were matched. 

In [ ]:
pargeocd = pd.read_csv('pargeocd.csv', sep = ";")
pargeocd = pargeocd.drop_duplicates('HANDLE')
final_map = matched.merge(pargeocd,left_on='HANDLE_M1', right_on='HANDLE', how='inner')
final_map.to_csv('final_map.csv')

    # Set ArcPy environment settings
arcpy.env.workspace = r'C:\Users\wyatt.clarke\Documents\ArcGIS\Projects\StLouis\StLouis.gdb'
arcpy.env.overwriteOutput = True
        #MakeXYEventLayer_management(in_Table, x_coords, y_coords, out_Layer, spRef, z_coords)
arcpy.management.MakeXYEventLayer(r"C:\Users\wyatt.clarke\Documents\Research\StLouis\StLouisAddressMatching_JupyterNotebook\final_map.csv", "XCoord", "YCoord", "final_map_Layer", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", None)
        #CopyFeatures_management (in_features, out_feature_class, {config_keyword}, {spatial_grid_1}, {spatial_grid_2}, {spatial_grid_3})
arcpy.management.CopyFeatures('final_map_Layer', 'final_map_Feature', None, None, None, None)

Gray dots are the parcels to which applicant addresses were matched. Blue dots are geocodes of parcels that could not be matched.
<img src="shapefiles/final2.PNG">

Left is a heatmap of postal addresses in St. Louis. Right is a heatmap of the postal addresses matched to an application.
<img src="shapefiles/allVSmatched_parcels_heatmaps2.PNG">